In [1]:
!pip3 install mnist
!pip3 install minisom

# imports
import matplotlib.pyplot as plt
import numpy as np
import mnist
import scipy.misc
import math
import seaborn as sns
import pickle

from matplotlib.colors import ListedColormap
from matplotlib import cm

from sklearn import datasets,metrics
from sklearn.datasets import fetch_20newsgroups, fetch_rcv1
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.decomposition import PCA, TruncatedSVD, KernelPCA
from sklearn.manifold import TSNE
import umap
from minisom import MiniSom


from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.cluster import KMeans, SpectralClustering, MiniBatchKMeans

from sklearn.metrics.pairwise import euclidean_distances

from joblib import Parallel, delayed
from tqdm import tqdm



!pip install ipython-autotime
%load_ext autotime

/home/adalmia1/anaconda3/lib/python3.7/site-packages/umap/__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")


time: 574 µs (started: 2021-02-08 09:16:45 -05:00)


In [2]:
# Load full MNIST dataset
MNIST_X_train = mnist.train_images()
MNIST_X_train = MNIST_X_train.reshape((len(MNIST_X_train), -1)) 
MNIST_y_train = mnist.train_labels()

MNIST_X_test = mnist.test_images()
MNIST_X_test = MNIST_X_test.reshape((len(MNIST_X_test), -1)) 
MNIST_y_test = mnist.test_labels()

total_samples = len(MNIST_X_train) + len(MNIST_X_test)


#Load the 20NewsGroups
vectorizer = CountVectorizer(min_df=5, stop_words='english') 

newsgroups_train = fetch_20newsgroups(subset='all')
newsgroups_X_train = pickle.load( open( "bert_20ng_everything.txt", "rb" ) )  #vectorizer.fit_transform(newsgroups_train.data).toarray()
newsgroups_y_train = newsgroups_train.target


print(newsgroups_X_train.shape)
print(newsgroups_y_train.shape)


(18846, 768)
(18846,)
time: 1.64 s (started: 2021-02-08 09:16:45 -05:00)


In [3]:
# k-Means clustering
def kmeans(X_train, n_clusters):
  kmeans = KMeans(n_clusters=n_clusters, random_state=2)
  y_pred_train = kmeans.fit_predict(X_train)
  train_centers = kmeans.cluster_centers_

  return y_pred_train
  

time: 468 µs (started: 2021-02-08 09:16:47 -05:00)


In [4]:
# Spectral clustering 
def spectral(X_train, n_clusters):
  #med = np.median(euclidean_distances(X_train)) gamma=2*(1/med)**2,
  spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', random_state=0)
  y_pred_train = spectral.fit_predict(X_train)

  return y_pred_train


time: 672 µs (started: 2021-02-08 09:16:47 -05:00)


In [5]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

# Evaluation function
def evaluate(X, y, y_pred):
    print("Accurary Score: %0.3f" % purity_score(y, y_pred))
    print("Adjusted Mutual Information Score: %0.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
    print("Adjusted Rand Index Score: %0.3f" % metrics.adjusted_rand_score(y, y_pred))
    print("Normalized Mutual Information Score: %0.3f" % metrics.normalized_mutual_info_score(y, y_pred))

    print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
    print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
    print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
    #print(np.unique(y_pred))
    print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, y_pred, sample_size=1000))
    print()



time: 922 µs (started: 2021-02-08 09:16:47 -05:00)


In [ ]:
# dimension reduction


def umap_reduce(X_train, dim):
  umapr = umap.UMAP(n_components=dim,  n_jobs=1)
  X_train_umap = umapr.fit_transform(X_train)
  #X_test_umap = umapr.fit_transform(X_test)
  return X_train_umap




def cv_score(estimator, g, X, y, n_clusters):
  estimator.set_params(**g)
  X_r = estimator.fit_transform(X)
  y_pred = kmeans(X_r, n_clusters)
  return (purity_score(y, y_pred), g)
  



def dimension_reduce(dimensions, X, y, n_clusters):
  #y_pred_train = kmeans(X, n_clusters)
  #print("Original KMeans")
  #evaluate(X,y,y_pred_train) 

  #y_pred_train = spectral(X, n_clusters)
  #print("Original Spectral")
  #evaluate(X ,y,y_pred_train)
  
  
  param_grid = [{
        "n_neighbors": [15, 25, 35, 50],
        "metric": ['euclidean', 'cosine', 'correlation', 'jaccard'],
        "min_dist": np.linspace(0.1, 1, 10)
    }]
  
  param_grid = ParameterGrid(param_grid)
  
  
  for dim in dimensions:
    
    umapr = umap.UMAP(n_components=dim, n_epochs = 500, n_jobs=1)
    print("Reduce")
    results = np.array(Parallel(n_jobs=4)(delayed(cv_score)(umapr, g, X, y, n_clusters)  for g in tqdm(param_grid))) 
    
    best_param = sorted(results, key=lambda t: t[0], reverse=True)[:1]
    best_param = best_param[0][1] 
    print(best_param)
    umapr.set_params(**best_param)
    X_umap = umapr.fit_transform(X)
    
    y_pred_train = kmeans(X_umap,  n_clusters)
    print("KMeans dimension reduce: " + str(dim))
    evaluate(X_umap,y,y_pred_train)

    #y_pred_train = spectral(X_umap, n_clusters)
    #print("Spectral dimension reduce: " + str(dim))
    #evaluate(X_umap,y,y_pred_train)




dimension_reduce([300, 500], newsgroups_X_train , newsgroups_y_train, 20)


  0%|          | 0/160 [00:00<?, ?it/s]

Reduce


 50%|█████     | 80/160 [3:11:04<3:06:45, 140.06s/it]/home/adalmia1/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████| 160/160 [10:52:39<00:00, 244.74s/it] 


{'metric': 'cosine', 'min_dist': 0.1, 'n_neighbors': 15}


In [ ]:
# Visualization general

def visualize(X_train, y_train, reduce, title, n_clusters):
 
  colors = []
  colors += cm.get_cmap("Set3").colors
  colors += cm.get_cmap("Set2").colors
  my_cmap = ListedColormap(colors)
  
  X_train_r = reduce(X_train, 2)
 

  kmy_pred_train, train_centers = kmeans(X_train_r, n_clusters)
  #spy_pred_train, spy_pred_test = spectral(X_train_r, X_test_r, y_train, y_test, n_clusters)

  
  fig=plt.figure(figsize=(8,5))
  # Train orginial labels
  plt.subplot(1, 2, 1)
  plt.gca().set_title('Train Data With Orginial Labels')
  plt.scatter(X_train_r[:, 0], X_train_r[:, 1], c= y_train, s=20, cmap=my_cmap)

  # Train kmeans labels
  plt.subplot(1, 2, 2)
  plt.gca().set_title('Train Data With KMeans Clusters')
  plt.scatter(X_train_r[:, 0], X_train_r[:, 1], c= kmy_pred_train, s=20, cmap=my_cmap)
  plt.scatter(train_centers [:, 0], train_centers [:, 1], c='black', s=100, alpha=0.8)


  fig.suptitle(title + " Reduction", fontsize=16)

  fig.tight_layout()
  fig.subplots_adjust(top=0.85)
  plt.show()



visualize(newsgroups_X_train , newsgroups_y_train,  umap_reduce, "UMAP BERT", 20)


In [ ]:
# Digits reduction 
#dimension_reduce([48, 32, 16, 8, 4, 2], digits_X_train, digits_X_test, digits_y_train, digits_y_test, kernel_pcarbf_reduce, 10)
#visualize(digits_X_train, digits_X_test, digits_y_train, digits_y_test, som_reduce, "SOM", 10)

#visualize(digits_X_train, digits_X_test, digits_y_train, digits_y_test, tsne_reduce, "TSNE", 10)

In [ ]:
# fetch 20 reduction 
#dimension_reduce([ 1000, 500, 300, 200, 100, 50, 20], newsgroups_X_train , newsgroups_X_test , newsgroups_y_train , newsgroups_y_test, 20)

#PCA
#visualize(newsgroups_X_train , newsgroups_X_test , newsgroups_y_train, newsgroups_y_test, pca_reduce, "PCA", 20)

In [ ]:
dimension_reduce([7500, 5000, 2500, 1000, 500, 250, 50, 20], newsgroups_X_train , newsgroups_X_test , newsgroups_y_train, newsgroups_y_test, umap_reduce, 20)

In [ ]:
dimension_reduce([7500, 5000, 2500, 1000, 500, 250, 50, 20], newsgroups_X_train , newsgroups_X_test , newsgroups_y_train, newsgroups_y_test, pca_reduce, 20)

In [ ]:
# rcv1 reduction
#dimension_reduce([ 1000, 500, 300, 200, 100, 50, 20], rcv1_X_train , rcv1_X_test , rcv1_y_train , rcv1_y_test, 103)
#visualize_pca(rcv1_X_train , rcv1_X_test , rcv1_y_train , rcv1_y_test, 103)
#visualize_tsne(rcv1_X_train , rcv1_X_test , rcv1_y_train , rcv1_y_test, 103, True)